In [ ]:
import requests
import json 
import psycopg2
from datetime import datetime
from datetime import timedelta
import string
import numpy
from re import sub
import pandas as pd
from bs4 import BeautifulSoup

import unittest


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/cs4111/cs4111_project/

In [ ]:
# Constant variables listing here!
API_KEY = 'API_KEY'

In [ ]:
# we should never write the pw in code! :(
conn = psycopg2.connect(dbname="project1", user="user", host="34.73.36.248", password="password")

In [ ]:
class Movie(object):
  DICT_GENRE = {}
  ALPHABET_USED_LIST = [0] * 26
  TABLE = "movie"

  def __init__(self, id: str, agelimit : str, summary : str, dateofrelease : str, 
                 duration : str, awards : str, genre : str, budget : int, profit : str):
    self.id = id
    self.agelimit = agelimit
    self.summary = summary
    self.dateofrelease = convert_date_format(dateofrelease)
    self.duration = convert_time_format(duration)
    self.awards = awards
    self.genresetid = create_genre_set_id(genre)
    self.budget = generate_budget()
    self.profit = profit


  def insert_movie_data(self):
    try:
      cur = conn.cursor()
      query = "INSERT INTO movie (id, agelimit, summary, dateofrelease, duration, awards, genresetid, budget, profit) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);"
      insert_data = (self.id, self.agelimit, self.summary, self.dateofrelease, self.duration, self.awards, self.genresetid, self.budget, self.profit)
      cur.execute(query, insert_data)
      conn.commit()
      cur.close()
      print("insert movie_id: " + self.id +  " data successfully!")

    except (Exception, psycopg2.DatabaseError) as error:
      print(error)
      conn.rollback()

  @staticmethod
  def insert_movie_data_batch(data):
    try:
      cur = conn.cursor()
      for item in data:
        id = item["imdbID"]
        agelimit = item["Rated"]
        summary = item["Plot"]
        dateofrelease = Movie.convert_date_format(item["Released"])
        duration = Movie.convert_time_format(item["Runtime"])
        awards = item['Awards']
        genresetid = Movie.create_genre_set_id(item["Genre"])
        # Movie.insert_genre_set(genresetid, item["Genre"])

        budget = Movie.generate_budget()
        profit = item["BoxOffice"]

        query = "INSERT INTO movie (id, agelimit, summary, dateofrelease, duration, awards, genresetid, budget, profit) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);"
        insert_data = (id, agelimit, summary, dateofrelease, duration, awards, genresetid, budget, profit)

        cur.execute(query, insert_data)
        conn.commit()
      cur.close()
      print("insert " + str(len(data)) + " data successfully!")

    except (Exception, psycopg2.DatabaseError) as error:
      print(error)
      conn.rollback()

  @staticmethod
  def get_id_batch():
    try:
      cur = conn.cursor()
      query = "SELECT id FROM movie;" 
      cur.execute(query)
      print("The number of parts: ", cur.rowcount)
      rows = cur.fetchall()
      cur.close()
      print("select " + str(len(rows)) + " data successfully!")
      # while row is not None:
      #   id_list.append(row)

    except (Exception, psycopg2.DatabaseError) as error:
      print(error)
      conn.rollback()

    return [rows[i][0] for i in range(len(rows))]

  # helper methods
  @staticmethod
  def convert_date_format(old_date):
    return datetime.strptime(old_date.replace(" ", "-"), '%d-%b-%Y').strftime('%Y-%m-%d')

  @staticmethod
  def convert_time_format(old_time):
    return str(timedelta(hours=0, minutes=int(old_time.split(' ')[0]), seconds=0)) 

  @staticmethod
  def create_genre_set_id(genre):
    genre_set = genre.split(',')
    genre_set_id = ''
    for g in genre_set:
      # if the gerne has not been store in the dictionary
      if not Movie.DICT_GENRE.get(g):
        # find a alphabet that has not been used by a genre
        idx = 0
        for i in range(26):
          if not Movie.ALPHABET_USED_LIST[i]:
            idx = i
            break

        Movie.DICT_GENRE[g] = string.ascii_lowercase[idx]
        genre_set_id += string.ascii_lowercase[idx]
        # assign the used alphabet to be 1
        Movie.ALPHABET_USED_LIST[idx] = 1
      #get the value to the genre
      else:
        genre_set_id += Movie.DICT_GENRE.get(g)
    return genre_set_id

  @staticmethod
  def insert_genre_set(genre_set_id, genre):
    genre_list = genre.split(',')
    for genre in genre_list:
      query = "INSERT INTO GenreSet VALUES (id, genre) (%s, %s);"
      insert_data = (genre_set_id, genre)


  @staticmethod
  def generate_budget():
  # $10 – $15 million
    return int(numpy.random.randint(low=10**7, high=1.5*10**7, size=1)[0])




In [ ]:
class Genre(object):
    dict_used = {}
    
    @staticmethod
    def insert_genre_set(movie_data):
      try:
        cur = conn.cursor()
        for item in movie_data:
          genre_list = item["Genre"].split(',')
          genre_set_id = Movie.create_genre_set_id(item["Genre"])
          for genre in genre_list:
            if not Genre.dict_used.get(genre_set_id + genre):
              query = "INSERT INTO GenreSet (id, genre) VALUES (%s, %s);"
              insert_data = (genre_set_id, genre)
              cur.execute(query, insert_data)
              Genre.dict_used[genre_set_id + genre] = 1
        cur.close()
        conn.commit()
        print("insert " + str(len(movie_data)) + " data successfully!")

      except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        conn.rollback()

In [ ]:
class Music(object):
    TABLE = "music"
    def __init__(self, id: str, name : str, singer : str, movie_id : str):
      self.id = id
      self.name = name
      self.singer = singer
      self.movie_id = movie_id

    @staticmethod
    def insert_music_data_batch(data):
      try:
        cur = conn.cursor()
        for i in range(len(data["id"])):
          id = data["id"][i]
          name = data["name"][i]
          singer = data["singer"][i]
          movie_id = data["movie_id"][i]

          query = "INSERT INTO " + Music.TABLE + " (id, name, singer, movie_id) VALUES (%s, %s, %s, %s);"
          insert_data = (id, name, singer, movie_id)

          cur.execute(query, insert_data)
          conn.commit()
        cur.close()
        print("insert " + str(len(data["id"])) + " data successfully!")

      except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        conn.rollback()




In [ ]:
class Scraper(object):

    def get_movie_data(self, imdb_id_set, apikey): 
      data = []
      for id in imdb_id_set:
        url = 'http://www.omdbapi.com/?i='+ str(id) + '&apikey=' + str(apikey)
        headers = {"Content_Type" : "application/json"}
        r = requests.get(url, headers = headers)
        if r.status_code == 200:
          data.append(json.loads(r.text))
        else:
          raise Exception("Response:" + str(r.status_code) + "to" + url)
      return data
    
    '''
    get music data of movies
    input: movie imdb_id_set, apikey
    output: dictionary with (id, name, singer, movie_id)
    '''
    def get_music_data(self, imdb_id_set): 
      data = {"id" : [], "name" : [], "singer" : [], "movie_id" : []}
      for id in imdb_id_set:
        url = 'https://www.imdb.com/title/' + str(id) +'/soundtrack?ref_=tt_trv_snd'
        headers = {"Content_Type" : "application/json"}
        r = requests.get(url, headers = headers)
        if r.status_code == 200:
          ids, names, singers, movie_ids = self.clean_music_data(r, id)
          data["id"] += ids
          data["name"] += names
          data["singer"] += singers
          data["movie_id"] += movie_ids
        else:
          raise Exception("Response:" + str(r.status_code) + "to" + url)
        
      return data

    def clean_music_data(self, r, movie_id):
      soup = BeautifulSoup(r.text, "html.parser")
      soundtrack_div = soup.find_all('div', class_='list')[0]

      # create lists to store music information in this movie (the reson why I dont use dict or df: inefficient append opearations)
      ids, names, singers, movie_ids = [], [], [], []
      
      for row in soundtrack_div:
        if len(row) > 10:
          ids.append(row.get('id')) #use get() to get the value inside a tag
          names.append(row.get_text().split('\n')[0])
          singer = row.find('a').get_text() if row.find('a') else 'N/A'
          singers.append(singer) # use find() to find the data that have tag 'a'
          movie_ids.append(movie_id)
      return ids, names, singers, movie_ids
        


In [ ]:
"""Be really careful with df.drop_duplicates(). 
The index will not be contiguous, which may lead to indexError when iterating it later"""

class Data_loader(object):
  def __init__(self):
    self.director_csv = 'Director.csv'
    self.actor_csv = 'Actor.csv'
    self.movie2actor_csv = 'm2a.csv'
    self.movie2director_csv = 'm2d.csv'

  def director(self):
    data = pd.read_csv(self.director_csv)
    data = data.drop(['Unnamed: 0'],axis=1)
    data = data.drop_duplicates()
    return data.values.tolist()

  def actor(self,):
    data = pd.read_csv(self.actor_csv)
    data = data.drop(['Unnamed: 0'],axis=1)
    data = data.drop_duplicates()
    return data.values.tolist()

  def movie2actor(self):
    data = pd.read_csv(self.movie2actor_csv)
    data = data.drop(['Unnamed: 0'],axis=1)
    data = data.drop_duplicates()
    return data.values.tolist()

  def movie(self):
    data = pd.read_csv(self.movie2actor_csv)
    data = data.drop(['Unnamed: 0'],axis=1)
    data = data['Movie_id']
    data = data.drop_duplicates()
    return data.values.tolist()

  def movie2director(self):
    data = pd.read_csv(self.movie2director_csv)
    data = data.drop(['Unnamed: 0'],axis=1)
    data = data.drop_duplicates()
    return data.values.tolist()



In [ ]:
          
class Test_Scraper(unittest.TestCase):
  def __init__(self):
    self.scraper = Scraper()
  
  def test_get_music_data(self):   
    data = self.scraper.get_music_data(["tt2948372", "tt4154796"])
    print(data)
    return data

  def test_clean_music_data(self):
    id = movie_id = "tt4154796"
    url = 'https://www.imdb.com/title/' + str(id) +'/soundtrack?ref_=tt_trv_snd'
    headers = {"Content_Type" : "application/json"}
    r = requests.get(url, headers = headers)
    # self.assertEqual(int(r.status_code), 200)
    data = self.scraper.clean_music_data(r, movie_id)
    print(data)

class Test_Movie(unittest.TestCase):
  # def __init__(self):
    # self.movie = Movie()

  def test_get_ids(self):
    ids = Movie.get_id_batch()
    print(ids)



In [ ]:

ts = Test_Scraper()
data = ts.test_get_music_data()
ts.test_clean_music_data()

tm = Test_Movie()
ids = tm.test_get_ids()

# if __name__ == "__main__":
#     unittest.main()


{'id': ['sn2213336', 'sn2213331', 'sn2213337', 'sn2213332', 'sn2213338', 'sn2213333', 'sn2213334', 'sn2213341', 'sn2217210', 'sn1860481', 'sn1861718', 'sn1861500', 'sn1861694', 'sn1861499', 'sn1861395'], 'name': ['We Get Along ', 'II B.S. ', 'Rappin Ced ', 'Subterranean Homesick Blues ', 'Check the Rhime ', 'Body and Soul ', 'Apple Tree ', "It's All Right ", "It's All Right (Duet version) ", 'Dear Mr. Fantasy ', 'Supersonic Rocket Ship ', 'Doom and Gloom ', 'Hey Lawdy Mama ', 'Come and Get Your Love ', "It's Been a Long, Long Time "], 'singer': ['Joe Crispiano', 'Charles Mingus', 'Daveed Diggs', 'Bob Dylan', 'Roger Ball', 'Johnny Green', 'Erykah Badu', 'Curtis Mayfield', 'Curtis Mayfield', 'Steve Winwood', 'Ray Davies', 'Mick Jagger', 'John Kay', 'Lolly Vegas', 'Sammy Cahn'], 'movie_id': ['tt2948372', 'tt2948372', 'tt2948372', 'tt2948372', 'tt2948372', 'tt2948372', 'tt2948372', 'tt2948372', 'tt2948372', 'tt4154796', 'tt4154796', 'tt4154796', 'tt4154796', 'tt4154796', 'tt4154796']}


In [ ]:
# get top 1000 movies' id from cvs files
loader = Data_loader()
id_list = loader.movie()
# get full movies' information
scraper = Scraper()
movie_data = scraper.get_movie_data(id_list, API_KEY)
# insert data to database
# Movie.insert_movie_data_batch(movie_data)
Genre.insert_genre_set(movie_data)
    

In [ ]:
# insert music data
movie_ids = Movie.get_id_batch()
scraper = Scraper()
music_info = scraper.get_music_data(movie_ids)
len(music_info["id"]), len(movie_ids)
Music.insert_music_data_batch(music_info)

The number of parts:  50
select 50 data successfully!


In [ ]:
#  sql query and operation
def select_all(table_name):
  try:
    cur = conn.cursor()
    cur.execute("SELECT * FROM " + table_name)
    print("The number of parts: ", cur.rowcount)
    row = cur.fetchone()
    cur.close()
    print("select " + str(len(row)) + " data successfully!")
    while row is not None:
      print(row)
  except (Exception, psycopg2.DatabaseError) as error:
    print(error)
    conn.rollback()

def create_table():
  try:
    cur = conn.cursor()
    queries = "create TABLE GenreSet (id varchar(26), genre varchar(100), PRIMARY KEY(id, genre));"
          
    for q in queries:
      cur.execute(q)
    # close communication with the PostgreSQL database server
    cur.close()
    # commit the changes
    conn.commit()
  except (Exception, psycopg2.DatabaseError) as error:
      print(error)
      conn.rollback()
  


In [ ]:
select  movie.* , music.name, sp.streamname
from movie, music, streamon so, streamingplatform sp, users, likingactor la, likingdirector ld, actingin, directingin 
where 
(la.peopleid = '2'
and actingin.peopleid = la.actorid
and movie.id = actingin.movieid)
or
(ld.peopleid = '2'
and directingin.peopleid = ld.dirid
and movie.id = directingin.movieid)

and movie.id = music.movie_id
and movie.id= so.movieid
and sp.streamid = so.streamid


In [ ]:

# conn.rollback()


In [ ]:


# conn.close()

In [ ]:
select actor.name, actor.awards
from (
    select la.actorid, max(count_users)
    from (
        select la.actorid, count(*) count_users
        from likingactor la
        group by la.actorid)) as sub, actor
      where sub.actorid = actor.peopleid


In [ ]:
select director.name, director.awards
from (
    select ld.dirid, max(count_users)
    from (
        select ld.dirid, count(*) count_users
        from likingdirector ld
        group by ld.dirid)) as sub, director
      where sub.dirid = director.peopleid

In [ ]:
select movie.*
from (
select w.movieid, max(count_users)
    from (
        select w.movieid, count(*) count_users
        from watchlist w
        group by w.movieid)) as sub, moive
        where movie.id = sub.movieid


In [ ]:
select distinct actors.name
from actors, actingin,
(select di.movieid
from directingin di, directors d
where d.peopleid = di.peopleid and d.name like '%Anthony Russo%') am
where actors.peopleid = actingin.peopleid and actingin.movieid = am.movieid;
